In [2]:
from bs4 import BeautifulSoup
import requests

from pandas import DataFrame as pd


PAGE = "https://scrapingclub.com"
PREF = "/exercise/list_basic/?page=1"
HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}


################## With tags ##################
def getSoup(page = PAGE, pref = PREF):
    req = requests.get(page + pref, headers = HEADERS)
    soup = BeautifulSoup(req.text, "html.parser")
    return soup


def getNavHrefs(soup):
    navHrefs = [PREF]
    soup = getSoup()

    navigateBar = soup.find("ul", attrs={"class":"pagination"})
    navButtons = navigateBar.find_all(lambda tag:tag.name == "a" \
                                      and "page" in tag["href"]  \
                                      and tag.text != "Next")
    for button in navButtons:
        navHrefs.append("/exercise/list_basic/" + button["href"])
    
    return navHrefs


def getInfFromCards(soup):
    itemsHrefs = []

    container = soup.find("div", attrs={"class" : "row my-4"})
    names = container.find_all("h4")
    prices = container.find_all("h5")

    for item in range(len(names)):
        name = names[item].text.replace("\n", "")
        price = prices[item].text
        
        itemsHrefs.append({
            "name" : name,
            "price" : price,
        })
    return itemsHrefs


################## With CSS selectors ##################
def getSoupCSS(page = PAGE, pref = PREF):
    req = requests.get(page + pref)
    soup = BeautifulSoup(req.text, "html.parser")
    return soup
    

def getNavHrefsCSS(soup):
    navHrefs = [PREF]
    soup = getSoup()

    navigateBar = soup.select("a.page-link")

    for i in navigateBar:
        navHrefs.append("/exercise/list_basic/" + i["href"])
    return navHrefs


def getInfFromCardsCSS(soup):
    itemsHrefs = []

    names = soup.select("h4.card-title")
    prices = soup.select("h5")

    for item in range(len(names)):
        itemsHrefs.append({
            "name" : (names[item].text).replace("\n", ""),
            "price" : prices[item].text
        })
    return itemsHrefs


def toPD(nav, func):
    """Проходит по списку ссылок nav, собирает Pandas DataFrame"""
    dfCards = pd(columns=["name", "price",])

    for i in nav:
        soup = getSoup(page = PAGE, pref = i)
        cards = func(soup)
        for card in cards:
            dfCards = dfCards.append(card, ignore_index = True)

    dfCards.to_csv(f"./result_docs/{func.__name__}.csv", index=True)
    dfCards.to_excel(f"./result_docs/{func.__name__}.xlsx", index=True)

    return dfCards

1. Поиск по тегам

In [6]:
soup = getSoup()
nav = getNavHrefs(soup = soup)

df = toPD(nav = nav, func = getInfFromCards)

print(df[:5])

                     name   price
0             Short Dress  $24.99
1        Patterned Slacks  $29.99
2     Short Chiffon Dress  $49.99
3  Off-the-shoulder Dress  $59.99
4              V-neck Top  $24.99


2. Поиск по селекторам

In [7]:
soupCSS = getSoupCSS()
navCSS = getNavHrefsCSS(soup = soupCSS)

dfCSS = toPD(nav = navCSS, func = getInfFromCardsCSS)

print(dfCSS[:5])

                     name   price
0             Short Dress  $24.99
1        Patterned Slacks  $29.99
2     Short Chiffon Dress  $49.99
3  Off-the-shoulder Dress  $59.99
4              V-neck Top  $24.99
